In [13]:
from time import strftime
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cross_validation, metrics, decomposition, neighbors
print strftime("%Y-%m-%d %X"), "clock set"

2015-04-30 10:08:01 clock set


In [29]:
# load X_given and y_given
with open('../data/train.csv') as f:
    given_csv = f.readlines()
    # number of columns in each row is 1 + 28^2 = 1 + 784
    given_csv = [np.array(map(int, x.split(','))) for x in given_csv[1:]]
    X_given = [x[1:] for x in given_csv]
    y_given = [x[0] for x in given_csv]
print strftime("%Y-%m-%d %X"), "given data loaded"

# load X_test
with open('../data/test.csv') as f:
    test_csv = f.readlines()
    # number of columns in each row is 28^2 = 784
    X_test = [np.array(map(int, x.split(','))) for x in test_csv[1:]]
print strftime("%Y-%m-%d %X"), "testing data loaded"

X_train, X_validate, y_train, y_validate = cross_validation.train_test_split(X_given, y_given, test_size=0, random_state=0)
print strftime("%Y-%m-%d %X"), "cross-validation loaded"

2015-04-30 10:43:40 given data loaded
2015-04-30 10:44:02 testing data loaded
2015-04-30 10:44:02 cross-validation loaded


In [32]:
# pca65 = decomposition.RandomizedPCA(n_components=65, whiten=False)
# X_transform65 = pca65.fit_transform(X_train)
# print strftime("%Y-%m-%d %X"), "PCA finished"
# knn1 = neighbors.KNeighborsClassifier(n_neighbors=1)
# knn1.fit(X_transform65, y_train)
# print strftime("%Y-%m-%d %X"), "knn finished"

In [33]:
# z = knn1.predict(pca65.transform(X_validate))
# print metrics.accuracy_score(y_validate, z)
# print strftime("%Y-%m-%d %X"), "finished scoring"

In [34]:
X_train_expanded = list(X_train)
y_train_expanded = list(y_train)
X_train_expanded += (np.reshape([list(row[1:28]) + [0] for row in np.reshape(i, [28, 28])], 28*28) for i in X_train)
y_train_expanded += y_train
print strftime("%Y-%m-%d %X"), "added shift left data"
X_train_expanded += (np.reshape([[0] + list(row[0:27]) for row in np.reshape(i, [28, 28])], 28*28) for i in X_train)
y_train_expanded += y_train
print strftime("%Y-%m-%d %X"), "added shift right data"


2015-04-30 10:45:22 added shift left data
2015-04-30 10:45:35 added shift right data


In [35]:
X_train_shift_up = [np.reshape(np.concatenate([np.reshape(i, [28, 28])[1:28], [np.reshape(X_train[0], [28, 28])[0]]]), -1) for i in X_train]
X_train_shift_down = [np.reshape(np.concatenate([[np.reshape(i, [28, 28])[27]], np.reshape(X_train[0], [28, 28])[0:27]]), -1) for i in X_train]

X_train_expanded += X_train_shift_up
y_train_expanded += y_train
print strftime("%Y-%m-%d %X"), "added shift up data"
X_train_expanded += (np.reshape([list(row[1:28]) + [0] for row in np.reshape(i, [28, 28])], 28*28) for i in X_train_shift_up)
y_train_expanded += y_train
print strftime("%Y-%m-%d %X"), "added shift left data"
X_train_expanded += (np.reshape([[0] + list(row[0:27]) for row in np.reshape(i, [28, 28])], 28*28) for i in X_train_shift_up)
y_train_expanded += y_train
print strftime("%Y-%m-%d %X"), "added shift right data"

X_train_expanded += X_train_shift_down
y_train_expanded += y_train
print strftime("%Y-%m-%d %X"), "added shift down data"
X_train_expanded += (np.reshape([list(row[1:28]) + [0] for row in np.reshape(i, [28, 28])], 28*28) for i in X_train_shift_down)
y_train_expanded += y_train
print strftime("%Y-%m-%d %X"), "added shift left data"
X_train_expanded += (np.reshape([[0] + list(row[0:27]) for row in np.reshape(i, [28, 28])], 28*28) for i in X_train_shift_down)
y_train_expanded += y_train
print strftime("%Y-%m-%d %X"), "added shift right data"


2015-04-30 10:45:38 added shift up data
2015-04-30 10:45:52 added shift left data
2015-04-30 10:46:05 added shift right data
2015-04-30 10:46:05 added shift down data
2015-04-30 10:46:18 added shift left data
2015-04-30 10:46:32 added shift right data


In [36]:
len(X_train), len(X_train_expanded), len(y_train_expanded)

(42000, 378000, 378000)

In [37]:
pca65 = decomposition.RandomizedPCA(n_components=65, whiten=False)
X_transform65 = pca65.fit_transform(X_train_expanded)
print strftime("%Y-%m-%d %X"), "PCA finished"
knn1 = neighbors.KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_transform65, y_train_expanded)
print strftime("%Y-%m-%d %X"), "knn finished"
# z = knn1.predict(pca65.transform(X_validate))
# print metrics.accuracy_score(y_validate, z)
# print strftime("%Y-%m-%d %X"), "finished scoring"

2015-04-30 11:24:31 PCA finished
2015-04-30 11:25:02 knn finished


In [38]:
z_test = knn1.predict(pca65.transform(X_test))
print strftime("%Y-%m-%d %X"), "predicted test data"


2015-04-30 11:35:09 predicted test data


In [39]:
with open('pca_knn_with_translation.csv', 'w') as f:
    f.write('ImageId,Label\n')
    for i in xrange(28000):
        f.write(str(i+1) + ',' + str(z_test[i]) + '\n')
